In [1]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1', 'Game2', 'Game3', 'Game4']
Opp = 'Heat'
Team = 'Celtics'

In [2]:
players = ['brogdon', 'brown', 'grant', 'horford', 'pritchard', 'rob', 'smart', 'tatum', 'white', 'team', 'opp']

In [4]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                
                # For game 1, set the new team file as the overall and continue to the opponent
                if game[0:5] == 'Game1':
                    ovr_player_data['team'] = newplayer
                    newplayer.to_csv(f'!Data/Team.csv') # Write the new team file to the overall data folder
                    continue
                    
                # Load overall data for the team
                ovrplayerFile = (f'!Data/Team.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                    
                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall data frames can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer # Reassign the ovr_data_player['team'] data to the updated data
                    ovrplayer.to_csv('!Data/Team.csv') # Write the new team file to the overall data folder
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Team file is handled, so continue to the opponent file

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                
                # For game 1, set the new opponent file as the overall and continue to the next player
                if game[0:5] == 'Game1':
                    ovr_player_data['opp'] = newplayer
                    newplayer.to_csv(f'!Data/Opp.csv') # Write the new opponent file into the overall data directory
                    continue
                    
                # Load overall data for the opponent
                ovrplayerFile = (f'!Data/Opp.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary

                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv('!Data/Opp.csv') # Write the updated opponent file to the overall data directory
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Opponent file is handled, continue and exit the loop
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
            
            # For game 1, set the new player file as the overall and continue to the next player
            if game == 'Game1':
                ovr_player_data[player] = newplayer
                newplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
                
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
            
            if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Grant file found for Game1.
No Pritchard file found for Game2.
No Pritchard file found for Game3.
No Pritchard file found for Game4.


In [5]:
newTatumStats = PPP(new_player_data['tatum'])
newTatumStats.to_csv(f'Tatum/PPP Stats/{prevGames[-1] + Opp}.csv')
newTatumStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.71,18.4,3,100.0,1.5,15.4,100.0,0,N/A,2,100.0,2.0,25.0,100.0,0,N/A,2,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,5.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,16.7,0.0,0,N/A,2,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.15,34.2,1,58.3,0.91,42.3,50.0,7,71.4,3,0.0,2.5,16.7,100.0,1,100.0,1,100.0
TRANSITION,1.0,5.3,0,50.0,2.0,3.8,100.0,1,100.0,0,N/A,0.0,8.3,0.0,0,N/A,1,0.0
Attacking Closeouts,1.78,23.7,1,75.0,1.4,19.2,75.0,2,100.0,2,50.0,2.25,33.3,75.0,0,N/A,4,75.0
Catch & Shoot,1.5,5.3,0,50.0,1.5,7.7,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [6]:
tatumStats = PPP(ovr_player_data['tatum'])
tatumStats.to_csv('!PPP Stats/tatumStats.csv')
tatumStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.23,25.7,7,61.5,1.19,16.8,63.6,7,71.4,4,50.0,1.27,47.0,60.0,5,100.0,10,40.0
PNR Screener,0.67,2.1,0,33.3,0.67,3.0,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.25,2.8,0,50.0,3.0,1.0,100.0,0,N/A,1,100.0,0.67,7.1,33.3,1,100.0,2,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.0,0.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.4,100.0,1,100.0,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.17,26.5,4,54.8,1.27,27.6,59.1,15,66.7,7,42.9,0.9,23.8,44.4,5,60.0,4,25.0
TRANSITION,1.11,13.4,1,42.9,1.0,16.1,41.7,9,55.6,3,0.0,1.74,6.9,50.0,0,N/A,2,50.0
Attacking Closeouts,1.13,18.1,4,50.0,0.82,20.8,40.0,8,62.5,7,14.3,2.4,11.9,80.0,0,N/A,5,80.0
Catch & Shoot,1.0,4.2,0,33.3,1.0,6.0,33.3,0,N/A,6,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [7]:
newBrownStats = PPP(new_player_data['brown'])
newBrownStats.to_csv(f'Brown/PPP Stats/{prevGames[-1] + Opp}.csv')
newBrownStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,12.5,0,33.3,0.0,6.2,0.0,1,0.0,0,N/A,1.5,25.0,50.0,1,0.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.68,24.5,0,40.0,0.68,36.8,40.0,5,40.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.07,35.2,0,50.0,1.65,34.0,80.0,4,100.0,1,0.0,0.0,37.5,0.0,0,N/A,3,0.0
Attacking Closeouts,2.0,4.2,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,12.5,100.0,1,100.0,0,N/A
Catch & Shoot,0.75,16.7,0,25.0,0.75,25.0,25.0,0,N/A,4,25.0,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
brownStats = PPP(ovr_player_data['brown'])
brownStats.to_csv('!PPP Stats/brownStats.csv')
brownStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.76,10.2,1,30.0,0.33,7.3,20.0,3,33.3,2,0.0,1.19,16.8,40.0,2,0.0,3,66.7
PNR Screener,3.0,0.9,0,100.0,3.0,1.2,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,3.0,0.9,0,100.0,3.0,1.2,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.67,2.6,0,33.3,0.67,3.7,33.3,3,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.62,30.1,3,33.3,0.81,28.4,47.4,16,50.0,3,33.3,0.25,33.9,9.1,4,25.0,7,0.0
TRANSITION,1.04,23.0,3,52.2,1.15,16.9,58.3,10,70.0,2,0.0,0.92,37.1,45.5,4,75.0,7,28.6
Attacking Closeouts,1.38,11.1,1,66.7,1.0,12.2,55.6,8,62.5,1,0.0,2.67,8.6,100.0,1,100.0,2,100.0
Catch & Shoot,0.21,12.0,0,7.1,0.21,17.1,7.1,0,N/A,14,7.1,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
newSmartStats = PPP(new_player_data['smart'])
newSmartStats.to_csv(f'Smart/PPP Stats/{prevGames[-1] + Opp}.csv')
newSmartStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,5.6,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.67,16.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.67,42.9,100.0,1,100.0,2,100.0
TRANSITION,2.03,19.1,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.03,49.1,100.0,3,100.0,0,N/A
Attacking Closeouts,0.0,16.7,0,0.0,0.0,27.3,0.0,1,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.29,38.9,0,42.9,1.29,63.6,42.9,0,N/A,7,42.9,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
smartStats = PPP(ovr_player_data['smart'])
smartStats.to_csv('!PPP Stats/smartStats.csv')
smartStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.15,19.0,1,50.0,0.82,14.4,25.0,2,0.0,2,50.0,1.33,23.1,62.5,5,60.0,3,66.7
PNR Screener,3.0,1.4,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,2.6,100.0,0,N/A,1,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,2.5,2.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.5,5.1,100.0,1,100.0,1,100.0
DBL Ball Handler,2.0,1.4,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.6,100.0,1,100.0,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.51,25.4,3,75.0,1.04,19.9,50.0,4,50.0,0,N/A,1.79,30.2,87.5,4,100.0,4,75.0
TRANSITION,1.2,19.5,3,77.8,1.06,11.1,100.0,1,100.0,0,N/A,1.25,26.8,75.0,6,100.0,2,0.0
Attacking Closeouts,0.52,5.3,0,0.0,0.52,11.4,0.0,1,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.15,17.8,0,38.5,1.25,35.3,41.7,0,N/A,12,41.7,0.0,2.6,0.0,0,N/A,1,0.0


In [11]:
newHorfordStats = PPP(new_player_data['horford'])
newHorfordStats.to_csv(f'Horford/PPP Stats/{prevGames[-1] + Opp}.csv')
newHorfordStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,10.0,0,100.0,2.0,14.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.0,20.0,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,66.7,50.0,1,100.0,1,0.0
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,60.0,0,50.0,1.5,85.7,50.0,0,N/A,6,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [12]:
horfordStats = PPP(ovr_player_data['horford'])
horfordStats.to_csv('!PPP Stats/horfordStats.csv')
horfordStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.5,10.3,1,33.3,1.0,7.7,50.0,2,50.0,0,N/A,0.0,15.4,0.0,0,N/A,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,2.6,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,7.7,0.0,0,N/A,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,2.6,0,100.0,2.0,3.8,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,5.1,0,50.0,1.0,7.7,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.57,19.6,0,60.0,0.0,3.8,0.0,0,N/A,1,0.0,1.81,51.1,75.0,3,100.0,1,0.0
Attacking Closeouts,1.67,7.7,0,66.7,2.0,3.8,100.0,1,100.0,0,N/A,1.5,15.4,50.0,0,N/A,2,50.0
Catch & Shoot,1.0,46.2,0,33.3,1.0,69.2,33.3,0,N/A,18,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [13]:
newBrogdonStats = PPP(new_player_data['brogdon'])
newBrogdonStats.to_csv(f'Brogdon/PPP Stats/{prevGames[-1] + Opp}.csv')
newBrogdonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,10.0,1,N/A,0.0,16.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,3.0,10.0,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,25.0,100.0,0,N/A,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,20.0,0,0.0,0.0,33.3,0.0,1,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.14,8.8,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,1.14,22.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.5,40.0,0,50.0,0.0,16.7,0.0,1,0.0,0,N/A,2.0,75.0,66.7,0,N/A,3,66.7
Catch & Shoot,0.0,10.0,0,0.0,0.0,16.7,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [14]:
brogdonStats = PPP(ovr_player_data['brogdon'])
brogdonStats.to_csv('!PPP Stats/brogdonStats.csv')
brogdonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.75,13.6,3,60.0,0.4,12.8,33.3,2,50.0,1,0.0,1.33,15.0,100.0,2,100.0,0,N/A
PNR Screener,3.0,1.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,5.0,100.0,0,N/A,1,100.0
DHO Ball Handler,1.5,3.4,0,50.0,0.0,2.6,0.0,0,N/A,1,0.0,3.0,5.0,100.0,0,N/A,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.15,11.7,0,0.0,0.0,10.3,0.0,3,0.0,1,0.0,0.35,14.4,0.0,0,N/A,2,0.0
TRANSITION,1.02,21.6,1,40.0,1.35,22.8,57.1,4,50.0,3,66.7,0.26,19.4,0.0,1,0.0,2,0.0
Attacking Closeouts,1.04,32.5,1,43.8,0.98,28.7,44.4,9,44.4,0,N/A,1.12,40.0,42.9,0,N/A,7,42.9
Catch & Shoot,0.0,13.6,0,0.0,0.0,17.9,0.0,0,N/A,7,0.0,0.0,5.0,0.0,0,N/A,1,0.0


In [15]:
newWhiteStats = PPP(new_player_data['white'])
newWhiteStats.to_csv(f'White/PPP Stats/{prevGames[-1] + Opp}.csv')
newWhiteStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,14.3,0,50.0,2.0,8.3,100.0,1,100.0,0,N/A,0.0,50.0,0.0,0,N/A,1,0.0
TRANSITION,1.0,21.4,1,50.0,0.0,16.7,0.0,1,0.0,0,N/A,3.0,50.0,100.0,0,N/A,1,100.0
Attacking Closeouts,2.27,6.3,0,N/A,2.27,7.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.29,50.0,0,42.9,1.29,58.3,42.9,0,N/A,7,42.9,N/A,0.0,N/A,0,N/A,0,N/A


In [16]:
whiteStats = PPP(ovr_player_data['white'])
whiteStats.to_csv('!PPP Stats/whiteStats.csv')
whiteStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.83,13.3,0,33.3,1.67,8.8,66.7,1,100.0,2,50.0,0.0,27.3,0.0,0,N/A,3,0.0
PNR Screener,3.0,2.2,0,100.0,3.0,2.9,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,8.9,0,50.0,1.0,5.9,50.0,2,50.0,0,N/A,1.0,18.2,50.0,1,100.0,1,0.0
TRANSITION,0.67,20.0,1,25.0,0.43,20.6,16.7,3,0.0,3,33.3,1.5,18.2,50.0,0,N/A,2,50.0
Attacking Closeouts,1.61,13.8,0,75.0,2.13,5.5,100.0,1,100.0,0,N/A,1.39,39.3,66.7,2,100.0,1,0.0
Catch & Shoot,1.5,40.0,1,52.9,1.5,52.9,52.9,0,N/A,17,52.9,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
newRobStats = PPP(new_player_data['rob'])
newRobStats.to_csv(f'Rob/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,28.6,1,0.0,0.0,33.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.6,26.9,0,100.0,1.6,31.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [18]:
robStats = PPP(ovr_player_data['rob'])
robStats.to_csv('!PPP Stats/robStats.csv')
robStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.11,38.3,4,83.3,1.45,35.7,83.3,6,83.3,0,N/A,0.0,50.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,3.0,0,100.0,2.0,3.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.6,5.7,0,100.0,1.6,7.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [19]:
newGrantStats = PPP(new_player_data['grant'])
newGrantStats.to_csv(f'Grant/PPP Stats/{prevGames[-1] + Opp}.csv')
newGrantStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,3.0,25.0,0,100.0,3.0,28.6,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,12.5,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,3.0,12.5,0,100.0,3.0,14.3,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.26,48.5,0,0.0,0.0,28.6,0.0,1,0.0,1,0.0,0.53,188.0,0.0,0,N/A,1,0.0
Catch & Shoot,3.0,12.5,0,100.0,3.0,14.3,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [20]:
grantStats = PPP(ovr_player_data['grant'])
grantStats.to_csv('!PPP Stats/grantStats.csv')
grantStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,3.0,12.3,0,100.0,3.0,8.3,100.0,0,N/A,4,100.0,3.0,33.3,100.0,0,N/A,3,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,3.5,0,0.0,0.0,4.2,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.67,15.8,0,33.3,0.67,18.8,33.3,9,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,3.0,3.5,0,100.0,3.0,4.2,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.02,24.1,0,50.0,1.2,20.8,60.0,8,75.0,2,0.0,0.53,41.8,0.0,0,N/A,2,0.0
Catch & Shoot,1.5,24.6,3,72.7,1.36,22.9,62.5,0,N/A,8,62.5,2.0,33.3,100.0,3,100.0,0,N/A


In [ ]:
newPritchardStats = PPP(new_player_data['pritchard'])
newPritchardStats.to_csv(f'Pritchard/PPP Stats/{newGame}.csv')
newPritchardStats

In [21]:
pritchardStats = PPP(ovr_player_data['pritchard'])
pritchardStats.to_csv('!PPP Stats/pritchardStats.csv')
pritchardStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.39,48.0,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,1.39,144.0,100.0,1,100.0,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,66.7,0,0.0,0.0,100.0,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [22]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.61,9.0,2,33.3,0.0,4.4,0.0,2,0.0,1,0.0,0.94,20.7,50.0,3,66.7,3,33.3
PNR Screener,1.14,5.5,1,50.0,1.14,7.7,50.0,2,50.0,4,50.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,3.0,1.6,0,100.0,3.0,1.1,100.0,0,N/A,1,100.0,3.0,2.8,100.0,0,N/A,1,100.0
DHO Screener,0.0,0.8,0,0.0,0.0,1.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.89,16.0,2,47.1,0.62,14.2,40.0,9,44.4,1,0.0,1.34,20.7,57.1,3,66.7,4,50.0
TRANSITION,1.3,19.4,1,61.9,1.33,15.7,66.7,10,70.0,2,50.0,1.26,28.7,55.6,4,100.0,5,20.0
Attacking Closeouts,0.73,14.0,1,26.7,0.34,13.1,10.0,5,20.0,5,0.0,1.53,16.3,60.0,1,100.0,4,50.0
Catch & Shoot,1.1,23.6,0,36.7,1.1,33.0,36.7,0,N/A,30,36.7,N/A,0.0,N/A,0,N/A,0,N/A


In [23]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.03,22.8,4,50.0,1.09,26.0,47.1,11,63.6,6,16.7,0.8,14.7,66.7,3,66.7,0,N/A
PNR Screener,1.29,5.9,2,80.0,1.0,4.7,66.7,3,66.7,0,N/A,1.67,8.8,100.0,1,100.0,1,100.0
DHO Ball Handler,0.61,4.1,0,25.0,1.0,3.5,33.3,0,N/A,3,33.3,0.0,5.5,0.0,0,N/A,1,0.0
DHO Screener,0.61,4.1,1,33.3,0.0,2.2,N/A,0,N/A,0,N/A,1.0,8.8,33.3,0,N/A,3,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.78,26.0,2,32.0,0.74,25.3,27.8,14,35.7,4,0.0,0.85,27.8,42.9,4,50.0,3,33.3
TRANSITION,1.2,13.3,3,75.0,1.4,9.3,71.4,5,100.0,2,0.0,1.0,23.5,80.0,4,100.0,1,0.0
Attacking Closeouts,1.27,6.6,2,80.0,1.36,6.9,75.0,3,100.0,1,0.0,1.0,5.9,100.0,1,100.0,0,N/A
Catch & Shoot,1.8,8.4,0,60.0,1.8,11.8,60.0,0,N/A,10,60.0,N/A,0.0,N/A,0,N/A,0,N/A


In [24]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.96,14.2,9,45.9,0.8,9.3,38.5,16,50.0,10,20.0,1.09,24.8,51.4,16,75.0,19,31.6
PNR Screener,1.38,5.6,5,68.2,1.48,6.0,66.7,10,80.0,8,50.0,1.12,4.7,75.0,0,N/A,4,75.0
DHO Ball Handler,2.33,1.1,0,83.3,2.25,1.1,75.0,0,N/A,4,75.0,2.5,1.2,100.0,1,100.0,1,100.0
DHO Screener,1.25,0.8,0,50.0,0.0,0.3,0.0,0,N/A,1,0.0,1.67,1.8,66.7,1,100.0,2,50.0
DBL Ball Handler,1.2,0.9,0,60.0,0.67,0.8,33.3,3,33.3,0,N/A,2.0,1.2,100.0,2,100.0,0,N/A
DBL Screener,2.0,0.4,0,100.0,2.0,0.6,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.9,19.2,11,43.2,0.94,16.8,46.8,39,46.2,8,50.0,0.85,24.4,38.2,16,62.5,18,16.7
TRANSITION,1.11,17.7,9,50.7,1.04,15.6,47.7,31,54.8,13,30.8,1.22,22.3,55.2,14,85.7,15,26.7
Attacking Closeouts,1.03,13.7,6,45.8,0.85,14.6,40.5,31,54.8,11,0.0,1.49,11.9,58.8,3,100.0,14,50.0
Catch & Shoot,0.86,16.2,2,29.8,0.87,22.9,29.6,0,N/A,81,29.6,0.67,1.8,33.3,1,100.0,2,0.0


In [25]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.15,21.4,11,54.7,1.26,20.7,54.1,42,59.5,19,42.1,0.93,22.9,56.0,16,68.8,9,33.3
PNR Screener,0.91,5.8,7,57.1,0.93,5.6,52.9,14,64.3,3,0.0,0.89,6.2,75.0,1,100.0,3,66.7
DHO Ball Handler,0.71,4.6,2,22.2,0.83,4.9,26.7,1,0.0,14,28.6,0.35,3.9,0.0,1,0.0,2,0.0
DHO Screener,0.92,4.2,1,29.4,0.53,1.1,0.0,0,N/A,1,0.0,1.01,11.6,31.2,1,0.0,15,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.03,28.7,11,47.1,0.86,29.3,38.4,74,43.2,12,8.3,1.45,27.3,69.7,16,81.2,17,58.8
TRANSITION,1.5,11.3,6,75.6,1.43,9.1,72.0,16,81.2,9,55.6,1.59,16.4,80.0,11,90.9,9,66.7
Attacking Closeouts,1.32,5.2,3,66.7,1.28,5.4,62.5,14,71.4,2,0.0,1.43,4.8,80.0,3,66.7,2,100.0
Catch & Shoot,1.82,10.0,1,63.8,1.78,13.3,62.2,2,100.0,43,60.5,2.43,2.0,100.0,0,N/A,2,100.0
